In [1]:
from pyspark import SparkContext, SparkConf, SparkFiles
# sc.stop()
# conf = SparkConf().set("spark.default.parallelism", 4).set('spark.driver.maxResultSize', '10G')
# sc =  SparkContext(conf=conf)

In [2]:
# PART1
# mapper1
def readFile(line):
    result = line.split(',')
    if (not result[0].isdigit()):
        return
    else:
        return int(result[1]), [[int(result[0]), result[2]]]
        # movieID, [userID, rating]

def reducer1(a, b):
    return a+b

def Calculate(line):
    # line = (movieID, [userID, rating])
    total = 0
    subMean = []
    subMeanSqr = 0
    merge = []
    for person in line[1]:
        total += float(person[1])
    mean = total / len(line[1])
    
    for person in line[1]:
        subMean.append(float(person[1]) - mean)
        subMeanSqr += pow(float(person[1]) - mean, 2)
    subMeanSqr = pow(subMeanSqr, 0.5)
    
    for i in range(len(line[1])):
        merge.append((line[1][i][0], [[line[0], subMean[i], subMeanSqr]]))
        # userID, [movieID, rating to this movie - mean, meanSquare]
    return merge

def Similarity(line):
    # line = userID, [ [movieID, rating-mean, meanSquare], [movieID, rating-mean, meanSquare], ... ]
    tmp = []
    for i in range(len(line[1])):
        for j in range(i + 1, len(line[1])):
            length = line[1][i][2] * line[1][j][2]
            if length == 0:
                tmp.append(((line[1][i][0], line[1][j][0]),[0]))
            else:
                tmp.append(((line[1][i][0], line[1][j][0]), [line[1][i][1] * line[1][j][1] / length]))
            # (moveID1, moveID2), [sim])
    return tmp


def Add(line):
    tmp = 0
    for sim in line[1]:
        tmp += sim
    return line[0], tmp

In [3]:
path = "smallData.csv"
file = sc.textFile(path)

m1 = file.map(readFile).filter(lambda x: x != None).reduceByKey(reducer1).sortByKey(True)

# for i in m1.collect():
#     print(i)

m2 = m1.flatMap(Calculate).reduceByKey(reducer1)
m3 = m2.flatMap(Similarity).reduceByKey(reducer1)


In [4]:
m3 = m3.map(Add).sortByKey(True)
m3 = m3.collect()
outputfile = open("OutputFile.txt", "w")
for i in range(len(m3)):
    outputfile.write("(" + str(m3[i][0][0]) + ', ' + str(m3[i][0][1]) + ') : ' + str(m3[i][1]))
    outputfile.write('\n')
outputfile.close()